# Bibliotecas

In [1]:
import os
import dotenv

import boto3
import pandas as pd
import numpy as np

from itertools import product # expand.grid
from pathlib import Path # salvar amostras
import openpyxl

import zipfile

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

True

# Funções

In [4]:
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

# 01) Importando dados

In [5]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
    )

def read_s3(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

# Importa o predict do bucket

In [6]:
dados_igr = read_s3(
    bucket_name=os.getenv("S3_BUCKET_NAME"), 
    folder_name=os.getenv("S3_FOLDER_NAME"),
    file_name='prime_k5_dez.csv')

# Importa dados do arquivo IGR feature store

In [12]:
print("Carregando dados do feature_store_igr")

dados_feat_store_igr = read_s3(
     bucket_name=os.getenv("S3_BUCKET_NAME"), 
     folder_name=os.getenv("S3_FOLDER_NAME"), 
     file_name='feature_store_igr_prime_ab1_k5_dez.csv')

Carregando dados do feature_store_igr


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

# Importa dados do arquivo do contribuinte feature store

In [10]:
print("Carregando dados do feature_store_contribuinte")

dados_contribuinte = read_s3(
    bucket_name=os.getenv("S3_BUCKET_NAME"), 
    folder_name=os.getenv("S3_FOLDER_NAME"), 
    file_name='feature_store_contribuinte_prime_k5_dez.csv') # Como podemos salvar? Adotar um padrão de nome p o arquivo

Carregando dados do feature_store_contribuinte


In [ ]:
print("Carregando dados de imovel e mercantil")
zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)

def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')

# 03) Filtro

In [48]:
dados_igr = dados_igr.loc[(dados_igr['da_aberto'] == 1)]

956966

--------------------------------------------------
# Momento em que começa o tratamento dos filtros para geração das listas

In [ ]:
# Colunas/variáveis que exportaremos no csv
col = ['cda',
       'id_pessoa',
       'atividade_principal',
       'idade_divida',
       'vlr_tributo',
       'vlr_taxa',
       'valor_total_da',
       'num_dist_cda',
       'quantidade_reparcelamento',
       'historico_pagamento_em_valor',
       'qtd_notas_2anos',
       'edificacao',
       'situacao',
       'situacao_cobranca',
       'endereco_existe',
       'cpf_cnpj_existe',
       'class_contribuinte_nome',
       'class_contribuinte_perfil',
       'igr',
       'rating_divida']

### Filtrando para manter: apenas registros de DAs abertas não ajuizadas

In [81]:
base_conjunta_aux = base_conjunta[['cda', 'tipo_divida', 'id_pessoa', 'da_aberto',
                                  'atividade_principal', 'situacao', 'tipo_tributo',  
                                   'vlr_tributo', 'vlr_taxa',
                                   'edificacao', 'cpf_cnpj_existe', 'protesto',
                                   'ajuizamento', 'refis', 
                                   'endereco_existe']]

chave = ['cda', 'tipo_divida', 'id_pessoa', 'da_aberto']

In [82]:
dados_igr3 = pd.merge(dados_igr, base_conjunta_aux, on = chave, how = "left")

956966

In [84]:
dados_contribuinte_aux = dados_contribuinte[['id_pessoa', 'tipo_divida', 'da_aberto', 'qtd_notas_2anos']]

Index(['id_pessoa', 'tipo_divida', 'da_aberto', 'qtd_notas_2anos'], dtype='object')

In [85]:
dados_igr3 = pd.merge(dados_igr3, dados_contribuinte_aux, on = ['tipo_divida', 'id_pessoa', 'da_aberto'], how = "left")

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso',
       'igr', 'class_contribuinte_nome', 'class_contribuinte_perfil',
       'rating_divida', 'atividade_principal', 'situacao', 'tipo_tributo',
       'vlr_tributo', 'vlr_taxa', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'endereco_existe', 'qtd_notas_2anos'],
      dtype='object')

In [87]:
dados_igr3 = dados_igr3[(dados_igr3['ajuizamento'] == 0)]

424105

### Filtrando para manter: apenas registros de DAs abertas não protestadas

In [89]:
dados_igr3 = dados_igr3[(dados_igr3['protesto'] == 0)]

336452

### Filtrando para manter: apenas registros de DAs abertas entre 0 e 3 anos 

In [91]:
dados_igr3 = dados_igr3[(dados_igr3['idade_divida'] >= 0) & (dados_igr3['idade_divida'] <= 3)]

231757

### Filtrando para manter: IGR classificou como de média, alta e altíssima prob de recuperação

In [93]:
valores_filtrar = ["MEDIA", "ALTA", "ALTISSIMA"]
dados_igr3 = dados_igr3[dados_igr3['rating_divida'].isin(valores_filtrar)]

117819

# 04) Separando Bases por Tipo de Tributo

In [99]:
dados_igr_imovel = dados_igr3[dados_igr3['tipo_divida'] == 'imovel'] 
dados_igr_merc = dados_igr3[dados_igr3['tipo_divida'] == 'mercantil'] 

## Imóvel

In [104]:
imovel_ajuizamento = dados_igr_imovel[dados_igr_imovel['valor_total_da'] >= 50000] 
imovel_protesto =  dados_igr_imovel[(dados_igr_imovel['valor_total_da'] >= 3000) & 
                                     (dados_igr_imovel['valor_total_da'] < 50000)]
imovel_negativar = dados_igr_imovel[(dados_igr_imovel['valor_total_da'] >= 1000) & 
                               (dados_igr_imovel['valor_total_da'] < 3000)] 

### entre 1.000 e 3.000

In [107]:
imovel_negativar = imovel_negativar.sort_values(by = "igr", ascending = False)
imovel_negativar = imovel_negativar[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\imovel_negativar.csv'
imovel_negativar.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

### Entre 3.000 e 50.000

In [114]:
imovel_protesto = imovel_protesto.sort_values(by = "igr", ascending = False)
imovel_protesto = imovel_protesto[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\imovel_protesto.csv'
imovel_protesto.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

64321191.44

### Acima de 50.000

In [120]:
imovel_ajuizamento = imovel_ajuizamento.sort_values(by = "igr", ascending = False)
imovel_ajuizamento = imovel_ajuizamento[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\imovel_ajuizamento.csv'
imovel_ajuizamento.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

14296760.129999999

## Mercantil

In [126]:
mercantil_ajuizar = dados_igr_merc[dados_igr_merc['valor_total_da'] >= 200000] 
mercantil_protestar =  dados_igr_merc[(dados_igr_merc['valor_total_da'] >= 3000) & (dados_igr_merc['valor_total_da'] < 200000)]
mercantil_negativar = dados_igr_merc[(dados_igr_merc['valor_total_da'] >= 1000) & (dados_igr_merc['valor_total_da'] < 3000)] 

## Entre 1000 e 3000

In [128]:
mercantil_negativar = mercantil_negativar.sort_values(by = "igr", ascending = False)
mercantil_negativar = mercantil_negativar[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\mercantil_negativacao.csv'
mercantil_negativar.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

## Entre 3.000 e 200.000

In [136]:
mercantil_protestar = mercantil_protestar.sort_values(by = "igr", ascending = False)
mercantil_protestar = mercantil_protestar[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\mercantil_protesto.csv'
igr_dagt03klt200k_media_alta_altissima_exportar.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

15581097.77

## Maior que 200.000

In [143]:
mercantil_ajuizar = mercantil_ajuizar.sort_values(by = "igr", ascending = False)
mercantil_ajuizar = mercantil_ajuizar[col]

caminho_arquivo_csv = r'C:\Users\BHN\datascience\divida_ativa\analysis\relacao_cdas_por_cenario_recuperacao\mercantil_ajuizamento.csv'
mercantil_ajuizar.to_csv(caminho_arquivo_csv, index=False, sep = ";", decimal = ",")

8916550.33